In [1]:
import requests

In [20]:
import json
from pandas.io.json import json_normalize

In [49]:
import re
from bs4 import BeautifulSoup

In [60]:
import pandas as pd 
import numpy as np 

import dataload

In [61]:
import warnings
warnings.filterwarnings(action='ignore')

In [62]:
buy_full = dataload.load_buy()
print(buy_full.shape)
buy_full.head()

(2056899, 6)


,date,sex,age,big_cat,sm_cat,qty
0,20180101,F,20,식품,가공란,37
1,20180101,F,30,식품,가공란,16
2,20180101,F,40,식품,가공란,9
3,20180101,F,50,식품,가공란,3
4,20180101,M,20,식품,가공란,13


In [65]:
buy_full.big_cat.value_counts()

식품       1229819
뷰티        697853
냉난방가전     129227
Name: big_cat, dtype: int64

In [68]:
food = list(buy_full[buy_full["big_cat"]=="식품"].sm_cat.unique())
beauty = list(buy_full[buy_full["big_cat"]=="뷰티"].sm_cat.unique())
app = list(buy_full[buy_full["big_cat"]=="냉난방가전"].sm_cat.unique())

In [55]:
url = 'https://datalab.naver.com/shoppingInsight/sKeyword.naver'

In [41]:
header = {
    'referer' : 'https://datalab.naver.com/shoppingInsight/sCategory.naver' , 
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
}

In [56]:
params = {
    'cid': '50000000',
    
    'timeUnit': 'date',
    'startDate': '2018-01-01', 
    'endDate': '2018-12-31',
    }

In [50]:
response = requests.post(url, headers=header)

In [57]:
response = requests.post(url, data = params, headers=header)

In [59]:
response.text

'<!doctype html><html lang="en"><head><title>HTTP Status 405 – Method Not Allowed</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 405 – Method Not Allowed</h1><hr class="line" /><p><b>Type</b> Status Report</p><p><b>Message</b> Request method \'POST\' not supported</p><p><b>Description</b> The method received in the request-line is known by the origin server but not supported by the target resource.</p><hr class="line" /><h3>Apache Tomcat/7.0.100</h3></body></html>'

In [39]:
json_normalize(response.json()["result"][0]["data"])

<ipython-input-39-f290f8098bd4>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(response.json()["result"][0]["data"])


,period,value
0,20180101,57.75037
1,20180102,59.97403
2,20180103,62.08791
3,20180104,61.66756
4,20180105,54.08339
...,...,...
360,20181227,71.68345
361,20181228,63.42447
362,20181229,67.23612
363,20181230,73.27083


In [40]:
response.json()

{'code': '',
 'message': '',
 'shoppingParam': {'startDate': '2018-01-01',
  'endDate': '2018-12-31',
  'timeUnit': 'date',
  'cid': '50000805',
  'device': '',
  'gender': '',
  'age': '',
  'keyword': None,
  'page': 0,
  'count': 0,
  'type': None,
  'date': None,
  'dateRange': '2018.01.01. ~ 2018.12.31.'},
 'chartType': 'lineChart',
 'range': '2018.01.01. ~ 2018.12.31.',
 'result': [{'code': '50000805',
   'title': '니트/스웨터',
   'fullTitle': '패션의류 > 여성의류 > 니트/스웨터',
   'data': [{'period': '20180101', 'value': 57.75037},
    {'period': '20180102', 'value': 59.97403},
    {'period': '20180103', 'value': 62.08791},
    {'period': '20180104', 'value': 61.66756},
    {'period': '20180105', 'value': 54.08339},
    {'period': '20180106', 'value': 61.21658},
    {'period': '20180107', 'value': 70.59588},
    {'period': '20180108', 'value': 66.00632},
    {'period': '20180109', 'value': 67.0957},
    {'period': '20180110', 'value': 64.08684},
    {'period': '20180111', 'value': 63.41386},
  

In [32]:
json_normalize(response.json()["result"][0]["data"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   period  365 non-null    object 
 1   value   365 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


<ipython-input-32-070414b2c6b1>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(response.json()["result"][0]["data"]).info()
